In [9]:
module LP

using NNLS
import Base: convert

type PNNLSProblem{T}
    A::Matrix{T}
    B::Matrix{T}
    c::Vector{T}
end

type LinearProgram{T}
    f::Vector{T}
    G::Matrix{T}
    W::Vector{T}
end

type NNLSProblem{T}
    A::Matrix{T}
    b::Vector{T}
end


function convert{T1, T2}(::Type{NNLSProblem{T1}}, p::PNNLSProblem{T2})
    A = convert(Matrix{T1}, p.A)
    B = convert(Matrix{T1}, p.B)
    c = convert(Vector{T1}, p.c)
    Abar = A - B * (B \ A)
    bbar = c - B * (B \ c)
    
#     Abar = (I - B * pinv(B)) * A
#     bbar = (I - B * pinv(B)) * c
    NNLSProblem{T1}(Abar, bbar)
end

convert{T}(::Type{NNLSProblem}, p::PNNLSProblem{T}) = convert(NNLSProblem{T}, p)

function convert{T1, T2}(::Type{PNNLSProblem{T1}}, lp::LinearProgram{T2})
    q, n = size(lp.G)
    A = [lp.W'       zeros(1, q)
         zeros(q, q) eye(q)
         lp.G'       zeros(n, q)]
    B = [lp.f'
         lp.G
         zeros(n, n)]
    c = vcat(0, lp.W, -lp.f)
    PNNLSProblem{T1}(A, B, c)
end

convert{T}(::Type{PNNLSProblem}, lp::LinearProgram{T}) = convert(PNNLSProblem{T}, lp)

convert{T1, T2}(::Type{LinearProgram{T1}}, lp::LinearProgram{T2}) = LinearProgram{T1}(lp.f, lp.G, lp.W)

function solve(p::NNLSProblem)
    work = NNLSWorkspace(p.A, p.b)
    nnls!(work)
    work.x, work.rnorm
end

function solve(p::PNNLSProblem)
    v, residual = solve(convert(NNLSProblem, p))
    u = -p.B \ (p.A * v - p.c)
#     u = -pinv(p.B) * (p.A * v - p.c)
    v, u, residual
end

function solve(p::LinearProgram)
    v, u, residual = solve(convert(PNNLSProblem, p))
    y = v[1:length(p.W)]
    s = v[(length(p.W) + 1):end]
    y, s, u, residual
end

function _solve_iterative(p::LinearProgram{BigFloat}, tolerance=1e-12)
    lastresidual = BigFloat(Inf)
    precision = 100
    while true
        y, s, u, residual = setprecision(BigFloat, precision) do
            solve(p)
        end
        @show residual
        is_feasible = residual < tolerance
        insufficient_progress = residual >= (3 / 4) * lastresidual
        at_iteration_limit = precision > 1000
        if is_feasible || insufficient_progress || at_iteration_limit
            return y, s, u, residual
        else
            precision += 100
            lastresidual = residual
        end
    end
end
                

function solve_iterative{T}(p::LinearProgram{T}, tolerance=1e-12)
    y, s, u, residual = solve(p)
    @show residual
    if residual < tolerance
        return y, s, u, residual, true
    else
        ybig, sbig, ubig, rbig = _solve_iterative(LinearProgram{BigFloat}(p), tolerance)
        return T.(ybig), T.(sbig), T.(ubig), T(rbig), rbig < tolerance
    end
end

end

LP

In [10]:
f = ones(2)
G = [1. 0
0 1
-1 0
0 -1]
W = [1., 1, 1, 1]
setprecision(BigFloat, 1000)
lp = LP.LinearProgram{BigFloat}(f, G, W)
LP.solve(lp)[4]

6.76238891255152873857181222036491570845801538217709647956340490140239445574487033475495579063507280410578642494803697286104758007481139852374636124350910917949146047378117333203822523110969941658557982415405524756253654566620926280079841931591323635388407938532860686740203958433928339404158572957419791e-301

In [11]:
LP.solve_iterative(LP.LinearProgram(f, G, W))[4]

residual = 2.2443315815740113e-16


2.2443315815740113e-16

In [12]:
f = ones(2)
G = [1. 0
0 1
-1 0]
W = [1., 1, 1]
lp = LP.LinearProgram(f, G, W)
LP.solve_iterative(lp)

residual = 1.0
residual = 1.0000000000000000000000000000000
residual = 1.0000000000000000000000000000000000000000000000000000000000000


([0.0,0.0,1.0],[2.0,1.0,0.0],[-1.0,1.55129e-60],1.0,false)

In [17]:
f = zeros(2)
G = [1. 0
0 1
-1 0
0 -1]
eps = 1e-12
W = [1., 1, -1 + eps, -1 + eps]
@show W
lp = LP.LinearProgram(f, G, W)
LP.solve_iterative(lp, 1e-16)

W = [1.0,1.0,-1.0,-1.0]
residual = 1.1102229668586038e-16
residual = 3.1554436208832528180195328169479e-30


([0.0,0.0,6.40163e-43,6.40163e-43],[9.99978e-13,9.99978e-13,0.0,0.0],[1.0,1.0],3.1554436208832527e-30,true)

In [36]:
f = ones(2)
G = [1. 0
0 1
-1 0
0 -1]
W = [1., 1, -1.0001, -1.0001]
lp = LP.LinearProgram(f, G, W)
LP.solve(lp)

work.rnorm = 0.00011180228089137442


([0.0,0.0,0.999975,0.999975],[0.0,0.0,0.0,0.0],[1.00006,1.00006])

In [37]:
using NPZ

In [38]:
data = npzread("near-infeasible-LP.npz")

Dict{String,Any} with 3 entries:
  "f" => [0.0; 0.0; … ; 0.0; 1.0]
  "A" => [0.0 0.0 … 0.0 -1.0; 0.0 0.0 … 0.0 -1.0; … ; 0.0 0.0 … 0.0 -1.0; 0.0 0…
  "b" => [0.497533; 0.497533; … ; 1.69995; 2.19993]

In [39]:
A = data["A"]
b = vec(data["b"])
f = vec(data["f"])

11-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 1.0

In [40]:
setprecision(BigFloat, 200)
lp = LP.LinearProgram{BigFloat}(f, A, b)
LP.solve(lp)

work.rnorm = 1.1868996206615984949165038426898774074846298227233919333951698e-50


(BigFloat[0.0000000000000000000000000000000000000000000000000000000000000,7.602000264210110672303619451645605803575126593089169913849916e-55,2.894915951086954154096234395521754431895309306576327270637971e-54,0.0000000000000000000000000000000000000000000000000000000000000,0.0000000000000000000000000000000000000000000000000000000000000,0.0000000000000000000000000000000000000000000000000000000000000,0.0000000000000000000000000000000000000000000000000000000000000,0.0000000000000000000000000000000000000000000000000000000000000,0.0000000000000000000000000000000000000000000000000000000000000,0.0000000000000000000000000000000000000000000000000000000000000  …  0.0000000000000000000000000000000000000000000000000000000000000,0.0000000000000000000000000000000000000000000000000000000000000,0.0000000000000000000000000000000000000000000000000000000000000,0.0000000000000000000000000000000000000000000000000000000000000,0.0000000000000000000000000000000000000000000000000000000000000,0.000000000000000000